In [1]:
import pymongo
from datetime import datetime, timedelta

In [2]:
pprint = lambda st : print(st.expandtabs(32))

In [11]:
client = pymongo.MongoClient()

pprint(f"HOST | PORT | address \t {client.HOST} | {client.PORT} | {client.address} ")
pprint(f"list_database_names \t {client.list_database_names()}")
db = client.test_database
pprint(f"test_database \t {client.test_database}")
pprint(f"test_collection \t {db.test_collection}")
pprint(f"list_database_names \t {db.list_collection_names()}")
pprint(f"command-dbstats \t {db.command('dbstats', 1024 * 1024)}")

HOST | PORT | address            localhost | 27017 | ('localhost', 27017) 
list_database_names              ['admin', 'config', 'local', 'test_database']
test_database                    Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'test_database')
test_collection                  Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'test_database'), 'test_collection')
list_database_names              ['posts']
command-dbstats                  {'db': 'test_database', 'collections': 1, 'views': 0, 'objects': 49, 'avgObjSize': 108.26530612244898, 'dataSize': 5305.0, 'storageSize': 36864.0, 'numExtents': 0, 'indexes': 1, 'indexSize': 36864.0, 'scaleFactor': 1.0, 'fsUsedSize': 229616046080.0, 'fsTotalSize': 250685575168.0, 'ok': 1.0}


In [17]:
print(client.server_info())


{'version': '4.2.8', 'gitVersion': '43d25964249164d76d5e04dd6cf38f6111e21f5f', 'modules': [], 'allocator': 'system', 'javascriptEngine': 'mozjs', 'sysInfo': 'deprecated', 'versionArray': [4, 2, 8, 0], 'openssl': {'running': 'Apple Secure Transport'}, 'buildEnvironment': {'distmod': '', 'distarch': 'x86_64', 'cc': '/Applications/Xcode10.2.0.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/clang: Apple LLVM version 10.0.1 (clang-1001.0.46.3)', 'ccflags': '-isysroot /Applications/Xcode10.2.0.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX10.14.sdk -mmacosx-version-min=10.12 -target darwin16.0.0 -arch x86_64 -fno-omit-frame-pointer -fno-strict-aliasing -ggdb -pthread -Wall -Wsign-compare -Wno-unknown-pragmas -Winvalid-pch -Werror -O2 -Wno-unused-local-typedefs -Wno-unused-function -Wno-unused-private-field -Wno-deprecated-declarations -Wno-tautological-constant-out-of-range-compare -Wno-tautological-constant-compare -Wno-tautological-unsigned-zero-c

In [107]:
post = {"author": "Mike",
        "tags": ["mongodb", "python", "pymongo"],
        "date": datetime.utcnow()}

pprint(f"inserted_id \t {db.posts.insert_one(post).inserted_id}")

pprint(f"count_documents \t {db.posts.count_documents({'author':'Mike'})}")
pprint(f"find_one \t {db.posts.find_one({'author':'Mike'})}")
pprint(f"Less than a minute")`
for post in posts.find({'date': {'$gt': datetime.utcnow() - timedelta(minutes=1)}, 'tags' : "python" }):
    pprint(f"\t {post}")

db.posts.create_index([('author', pymongo.ASCENDING)], unique=True)

inserted_id                      5f22313a1ab042556a28b0ef
count_documents                  48
find_one                         {'_id': ObjectId('5f22233f1ab042556a28b0bd'), 'author': 'Mike', 'likes': ['mongodb', 'python', 'pymongo'], 'date': datetime.datetime(2020, 7, 30, 1, 32, 47, 640000)}
Less than a minute
                                 {'_id': ObjectId('5f2231201ab042556a28b0ed'), 'author': 'Mike', 'tags': ['mongodb', 'python', 'pymongo'], 'date': datetime.datetime(2020, 7, 30, 2, 32, 0, 793000)}
                                 {'_id': ObjectId('5f22313a1ab042556a28b0ef'), 'author': 'Mike', 'tags': ['mongodb', 'python', 'pymongo'], 'date': datetime.datetime(2020, 7, 30, 2, 32, 26, 696000)}


DuplicateKeyError: E11000 duplicate key error collection: test_database.posts index: author_1 dup key: { author: "Mike" }

In [1]:
def db2dicts(db_path):
    # album_movie, broadcast_character
    db = bsddb3.btopen(db_path, 'r')
    cate = os.path.basename(db_path)[:-3]   # album.db -> album
    for id in db:
        try:
            dic = json.loads(db[id])
            assert isinstance(dic, dict)
        except:
            # db value in wrong format
            dic = {'raw': db[id].decode('utf-8')}

        dic.update({'id': id.decode('utf-8'), 'cate': cate})

        # check for invalid table column
        yield({'_' + k: v for k, v in dic.items()})

def db2server(db_path, db_name=None):
    cate = os.path.basename(db_path)[:-3]

    client = pymongo.MongoClient()
    db = client[mongo_db_name]

    for dic in db2dicts(db_path):
        try:
            db[cate].insert_one(dic)
        except:
            logger.warning("invalid" + str(sys.exc_info()[0]))